# Проблемы на данный момент
1) Необходимо что-то сделать с токенизатором или с изначальным разделением на токены так как есть проблемы с его обратным конвертированием
2) Модель чаще всего предсказывает куски, необходим алгоритм поиска или дообучение


In [1]:
import json
import re
import numpy as np
import pandas as pd
import ast
import random
from spacy.training import offsets_to_biluo_tags, biluo_to_iob
import spacy
from spacy.training.example import Example
from spacy.lang.ru import Russian
from pymystem3 import Mystem
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from datasets import load_dataset, load_metric
from datasets import Dataset, DatasetDict
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import AutoTokenizer
from transformers import DataCollatorForTokenClassification
from transformers import pipeline
import re

In [2]:
with open('dataset\\train.json', 'r', encoding='utf-8') as f: #открыли файл с данными
    data = json.load(f) #загнали все, что получилось в переменную
data[0]

{'id': 809436509,
 'text': 'Извещение о проведении открытого конкурса в электронной форме для закупки №0328300032822000806 Общая информация Номер извещения 0328300032822000806 Наименование объекта закупки Поставка продуктов питания Способ определения поставщика (подрядчика, исполнителя) Открытый конкурс в бль Порядок внесения денежных средств в качестве обеспечения заявки на участие в закупке, а также условия гарантии Обеспечение заявки на участие в закупке может предоставляться участником закупки в виде денежных средств или независимой гарантии, предусмотренной ст. 45 Федерального закона № 44-ФЗ. Выбор способа обеспечения осуществляется участником закупки самостоятельно. Срок действия независимой гарантии должен составлять не менее месяца с даты окончания срока подачи заявок. Обеспечение заявки на участие в закупке предоставляется в соответствии с ч. 5 ст. 44 Федерального закона № 44-ФЗ. Условия независимой гарантии в соответствии со ст. 45 Федерального закона № 44-ФЗ. Реквизиты счета

In [3]:
labels_ner = {'обеспечение исполнения контракта': 'EC', 'обеспечение гарантийных обязательств': 'PWO'}

def create_dataset(data):
    nlp = spacy.blank("ru")
    max_lengh_tag = 0
    count = 0
    ner_data = []
    unique_list = []

    for dict_data in data:
        #d = {'sentence_id': None, 'words': None, 'labels':None}
        doc = nlp(dict_data.get('text'))
        #for token in doc:
        #print([t.text for t in doc])
        #print('Длинна токена: ',len(doc))

        if len(doc) > 512:
            count+=1
        if max_lengh_tag < len(doc):
            max_lengh_tag = len(doc)

        label = labels_ner.get(dict_data.get('label'))
        answer_start = dict_data.get('extracted_part').get('answer_start')[0]
        answer_end = dict_data.get('extracted_part').get('answer_end')[0]

        entities = [(answer_start, answer_end, label)]
        tags = offsets_to_biluo_tags(doc, entities)
        iob_tags = list(biluo_to_iob(tags)) # Переводим в формат IOB

        if answer_start!=0 and answer_end!=0 and '-' not in iob_tags:
            words = [t.text for t in doc]
            ner_data.append({'tokens': words, 'tags': iob_tags})

            unique_list += iob_tags
            unique_list = list(set(unique_list))

    return ner_data, unique_list

''' Создаём список из всех уникальных тегов и необходимый формат данных'''
ner_data, tags_list = create_dataset(data)


D:\Jupiter\venv\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "РАЗДЕЛ 4. тельно); в) 1 процент цены контракта, ес..." with entities "[(1010, 1179, 'EC')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
D:\Jupiter\venv\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "PAGE ПРОЕКТ КОНТРАКТА КОНТРАКТ № ______ на поставк..." with entities "[(1226, 1355, 'EC')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
D:\Jupiter\venv\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "ПРОЕКТ Государственный контракт №________ на поста..." with ent

In [4]:
print(tags_list)

['B-PWO', 'O', 'I-EC', 'B-EC', 'I-PWO']


In [5]:
print(ner_data[0])

{'tokens': ['Извещение', 'о', 'проведении', 'открытого', 'конкурса', 'в', 'электронной', 'форме', 'для', 'закупки', '№', '0328300032822000806', 'Общая', 'информация', 'Номер', 'извещения', '0328300032822000806', 'Наименование', 'объекта', 'закупки', 'Поставка', 'продуктов', 'питания', 'Способ', 'определения', 'поставщика', '(', 'подрядчика', ',', 'исполнителя', ')', 'Открытый', 'конкурс', 'в', 'бль', 'Порядок', 'внесения', 'денежных', 'средств', 'в', 'качестве', 'обеспечения', 'заявки', 'на', 'участие', 'в', 'закупке', ',', 'а', 'также', 'условия', 'гарантии', 'Обеспечение', 'заявки', 'на', 'участие', 'в', 'закупке', 'может', 'предоставляться', 'участником', 'закупки', 'в', 'виде', 'денежных', 'средств', 'или', 'независимой', 'гарантии', ',', 'предусмотренной', 'ст.', '45', 'Федерального', 'закона', '№', '44-ФЗ', '.', 'Выбор', 'способа', 'обеспечения', 'осуществляется', 'участником', 'закупки', 'самостоятельно', '.', 'Срок', 'действия', 'независимой', 'гарантии', 'должен', 'составлять'

In [6]:
ner_train, ner_test = train_test_split(ner_data, test_size=0.2, random_state=12345)

ner_train = pd.DataFrame(ner_train)
ner_test = pd.DataFrame(ner_test)

print(ner_train.info())
print(ner_test.info())
display(ner_train)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1172 entries, 0 to 1171
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tokens  1172 non-null   object
 1   tags    1172 non-null   object
dtypes: object(2)
memory usage: 18.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294 entries, 0 to 293
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tokens  294 non-null    object
 1   tags    294 non-null    object
dtypes: object(2)
memory usage: 4.7+ KB
None


,tokens,tags
0,"[Приложение, №, 2, к, Извещению, ГОСУДАРСТВЕНН...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,"[1, 2, Утверждено, на, заседании, Постоянно, д...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,"[Извещение, о, проведении, электронного, аукци...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,"[Извещение, о, проведении, электронного, аукци...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,"[УТВЕРЖДАЮ, Директор, бюджетного, стационарног...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
...,...,...
1167,"[УТВЕРЖДАЮ, :, Главный, врач, ГАУЗ, ТО, «, ОЛР...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1168,"[Проект, договора, №, _, _, _, _, _, _, _, _, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1169,"[1, Приложение, №, 4, к, Извещению, об, электр...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1170,"[Извещение, о, проведении, электронного, аукци...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [7]:
ner_data = DatasetDict({
    'train': Dataset.from_pandas(ner_train),
    'test': Dataset.from_pandas(ner_test)
})
ner_data

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 1172
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 294
    })
})

In [8]:
model_checkpoint = "cointegrated/rubert-tiny"

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

<h1 color=red>Необходимо разобраться</h1>

In [10]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Специальные токены имеют идентификатор слова None. Мы устанавливаем метку на -100, чтобы они автоматически игнорировались в функции потерь.
            if word_idx is None:
                label_ids.append(-100)
            # Мы устанавливаем метку для первого токена каждого слова.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # Для других токенов в слове мы устанавливаем метку либо на текущую метку, либо на -100, в зависимости от флага label_all_tokens.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        label_ids = [tags_list.index(idx) if isinstance(idx, str) else idx for idx in label_ids]

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [11]:
tokenize_and_align_labels(ner_data['train'][1:2])

{'input_ids': [[2, 21, 22, 299, 11513, 18674, 14380, 548, 650, 2448, 3526, 1768, 1499, 13151, 5290, 948, 28580, 7084, 10208, 650, 24624, 22373, 11543, 16892, 332, 4307, 3683, 295, 20887, 105, 296, 4829, 22797, 613, 17193, 11281, 117, 17, 310, 25364, 11424, 27928, 422, 997, 733, 1004, 18, 1201, 18, 22285, 315, 18, 328, 12666, 3575, 1635, 1154, 1186, 13167, 7084, 10837, 330, 3091, 15577, 7113, 656, 778, 28048, 326, 650, 24624, 1550, 13, 16006, 293, 6325, 15014, 16, 16296, 22069, 1229, 1647, 13361, 25409, 314, 778, 3003, 21901, 326, 2225, 24967, 613, 650, 27353, 603, 3789, 679, 16, 548, 671, 320, 2445, 2225, 18583, 811, 18, 3086, 328, 4992, 17009, 626, 2225, 18583, 603, 329, 958, 6788, 871, 20670, 1512, 18836, 603, 20939, 8558, 5584, 10647, 3118, 3789, 1385, 12, 1395, 656, 6086, 16654, 13, 7857, 5968, 2225, 12666, 17557, 745, 13120, 2095, 10312, 705, 26941, 988, 24921, 12847, 16767, 656, 329, 331, 17009, 761, 4295, 10712, 811, 1781, 650, 1231, 21820, 775, 18, 29, 18, 22, 18, 282, 8896, 70

In [12]:
''' Создаём датасет и хешируем его '''
tokenized_datasets = ner_data.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/1172 [00:00<?, ? examples/s]

Map:   0%|          | 0/294 [00:00<?, ? examples/s]

In [13]:
tags_list

['B-PWO', 'O', 'I-EC', 'B-EC', 'I-PWO']

#### Создаём модель
Теперь, когда наши данные готовы, мы можем загрузить предварительно обученную модель и настроить ее. Поскольку все наши задачи связаны с классификацией токенов, мы используем класс AutoModelForTokenClassification. Как и в случае с токенизатором, метод from_pretrained загрузит и кэширует модель для нас.
Единственное, что нам нужно указать, — это количество тегов (IOB) для нашей задачи

In [14]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(tags_list))
model.config.id2label = dict(enumerate(tags_list))
model.config.label2id = {v: k for k, v in model.config.id2label.items()}

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized f

In [21]:
batch_size = 16

args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=30,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
)

PyTorch: setting up devices


##### Пояснение к коду ниже
Затем нам понадобится средство подборки данных, которое объединит наши обработанные примеры вместе, применяя заполнение, чтобы сделать их все одинакового размера (каждое заполнение будет дополнено до длины самого длинного примера). Для этой задачи в библиотеке Transformers есть "сопоставитель данных", который дополняет не только входные данные, но и метки:

In [22]:
data_collator = DataCollatorForTokenClassification(tokenizer)

#### Загружаем метрику для прогноза
Здесь мы загрузим метрику seqeval (которая обычно используется для оценки результатов в наборе данных CONLL) через библиотеку наборов данных.

In [23]:
metric = load_metric("seqeval")

In [24]:
example = ner_train.to_dict('records')[1]
labels = example['tags']

metric.compute(predictions=[labels], references=[labels])

{'EC': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

Таким образом, нам нужно будет провести небольшую постобработку наших прогнозов:

* выбираем прогнозируемый индекс (с максимальным логитом) для каждого токена
* преобразуем его в строковую метку
* игнорируем везде, где мы устанавливаем метку -100

Следующая функция выполняет всю эту постобработку результата Trainer.evaluate (который представляет собой namedtuple, содержащий предсказания и метки) перед применением метрики:

In [25]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Удалить игнорируемый индекс (специальные токены)
    true_predictions = [
        [tags_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    true_labels = [
        [tags_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels, zero_division=0)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [26]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [27]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, tags. If tokens, tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 294
  Batch size = 16
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 1.4011664390563965,
 'eval_precision': 0.0018420529861123652,
 'eval_recall': 0.11929824561403508,
 'eval_f1': 0.003628085651276944,
 'eval_accuracy': 0.5384502455690796,
 'eval_runtime': 4.4228,
 'eval_samples_per_second': 66.474,
 'eval_steps_per_second': 4.296}

In [28]:
# разморозка
for param in model.parameters():
    param.requires_grad = True

In [29]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, tags. If tokens, tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
D:\Jupiter\venv\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1172
  Num Epochs = 30
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2220
  Number of trainable parameters = 11688077


Epoch,Training Loss,Validation Loss


The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, tags. If tokens, tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 294
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, tags. If tokens, tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 294
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, tags. If tokens, tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 294
  Ba

TrainOutput(global_step=2220, training_loss=0.09945100964726629, metrics={'train_runtime': 347.831, 'train_samples_per_second': 101.084, 'train_steps_per_second': 6.382, 'total_flos': 248830567096320.0, 'train_loss': 0.09945100964726629, 'epoch': 30.0})

In [30]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: tokens, tags. If tokens, tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 294
  Batch size = 16


{'eval_loss': 0.04696035012602806,
 'eval_precision': 0.3514376996805112,
 'eval_recall': 0.6432748538011696,
 'eval_f1': 0.4545454545454546,
 'eval_accuracy': 0.9835642216527867,
 'eval_runtime': 2.7084,
 'eval_samples_per_second': 108.551,
 'eval_steps_per_second': 7.015,
 'epoch': 30.0}

In [31]:
model.save_pretrained('ner_bert_30.bin')
tokenizer.save_pretrained('ner_bert_30.bin')

Configuration saved in ner_bert_30.bin\config.json
Model weights saved in ner_bert_30.bin\pytorch_model.bin
tokenizer config file saved in ner_bert_30.bin\tokenizer_config.json
Special tokens file saved in ner_bert_30.bin\special_tokens_map.json


('ner_bert_30.bin\\tokenizer_config.json',
 'ner_bert_30.bin\\special_tokens_map.json',
 'ner_bert_30.bin\\vocab.txt',
 'ner_bert_30.bin\\added_tokens.json',
 'ner_bert_30.bin\\tokenizer.json')

In [32]:
#pipe = pipeline(model=model, tokenizer=tokenizer, task='ner', aggregation_strategy='average', device=0)
pipe = pipeline(model='ner_bert_30.bin', tokenizer='ner_bert_30.bin', task='ner', aggregation_strategy='average', device=0)

loading configuration file ner_bert_30.bin\config.json
Model config BertConfig {
  "_name_or_path": "ner_bert_30.bin",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "emb_size": 312,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 312,
  "id2label": {
    "0": "B-PWO",
    "1": "O",
    "2": "I-EC",
    "3": "B-EC",
    "4": "I-PWO"
  },
  "initializer_range": 0.02,
  "intermediate_size": 600,
  "label2id": {
    "B-EC": 3,
    "B-PWO": 0,
    "I-EC": 2,
    "I-PWO": 4,
    "O": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 3,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 29564
}

loading configuration file ner_b

In [36]:
def preprocessing_pred(pred):
    pred = max([elem.get('word') for elem in pred], key=len)
    #print(pred)
    pred = pred.replace('( ', '(')
    pred = pred.replace('_ ', '_')
    pred = pred.replace(' )', ')')

    #print('Нужная строка: ', entit)
    splt = pred.split()
    pattern = splt[0]+'.{1,}'+splt[-1]
    try:
        pred = re.search(pattern, data[i]['text'])[0]
    except:
        print('Предсказанная строка: ', pred)
        print('Нужная строка: ', entit)
        print(re.search(pattern, data[i]['text']))
    return pred

In [37]:

count_true = 0
for i in range(len(data)):
    text = data[i]['text']
    entit = data[i]['extracted_part'].get('text')
    if entit:
        entit = entit[0]
    else:
        entit =''

    if pipe(text):
        #print(pipe(text))
        #pred = str(pipe(text)[0].get('word'))
        #print('Предсказанная строка: ', pred)
        pred = preprocessing_pred(pipe(text))
        #print(pattern)
        #print('После поиска: ', pred, end='\n\n')
    else:
        pred = ''


    if entit == pred:
        count_true+=1
print(count_true/len(data))


Предсказанная строка:  Размер
Нужная строка:  Размер обеспечения исполнения контракта 10.00%
None
Предсказанная строка:  Размер обеспечения исполнения контракта составляет 10 (Десять) процентов цены контракта, что составляет _____рублей.
Нужная строка:  Размер обеспечения исполнения контракта составляет 10 (Десять) процентов цены контракта, что составляет _____ рублей.
None
Предсказанная строка:  Обеспечение исполнения Контракта устанавливается в размере 0, 5 % от цены Контракта, что составляет ___________(____________)
Нужная строка:  Обеспечение исполнения Контракта устанавливается в размере 0,5 % от цены Контракта, что составляет ___________ (____________
None
Предсказанная строка:  Обеспечение исполнения контракта установлено в размере 5, 00 % от цены контракта на сумму _________рублей
Нужная строка:  Обеспечение исполнения контракта установлено в размере 5,00% от цены контракта на сумму _________ рублей.
None
Предсказанная строка:  Размер обеспечения исполнения Контракта составляе

error: missing ), unterminated subpattern at position 11

In [13]:
s = 'Размер обеспечения исполнения контракта устанавливается в размере 5 ( пять ) процентов от цены, по которой заключается контракт и составляет _ _ _ _ _ _ _ _ _ ( _ _ _ _ _'

s = 'Размер обеспечение исполнения контракта устанавливается от 0, 5 % до 30 % начальной ( максимальной ) цены контракта'

s = 'Размер обеспечения исполнения контракта 6593. 25 Российский рубль'
s = s.replace('( ', '(')
s = s.replace('_ ', '_')
s = s.replace(' )', ')')

print(s)
splt = s.split()
pattern = splt[0]+'.{1,}'+splt[-1]
print(pattern)
pred = re.search(pattern, data[0]['text'])
print(pred[0])
print(pred.span())

Размер обеспечения исполнения контракта 6593. 25 Российский рубль
Размер.{1,}рубль
Размер обеспечения исполнения контракта 6593.25 Российский рубль
(1279, 1343)


RuntimeError: The size of tensor a (823) must match the size of tensor b (512) at non-singleton dimension 1

### Загрузка модели

In [62]:
pip_2 = pipeline(task = 'ner', model='ner_bert_20.bin', tokenizer='ner_bert_tok.bin', aggregation_strategy='average', device=0)

loading configuration file ner_bert_20.bin\config.json
Model config BertConfig {
  "_name_or_path": "ner_bert_20.bin",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "emb_size": 312,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 312,
  "id2label": {
    "0": "I-PWO",
    "1": "O",
    "2": "B-PWO",
    "3": "I-EC",
    "4": "B-EC"
  },
  "initializer_range": 0.02,
  "intermediate_size": 600,
  "label2id": {
    "B-EC": 4,
    "B-PWO": 2,
    "I-EC": 3,
    "I-PWO": 0,
    "O": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 3,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 29564
}

loading configuration file ner_b

In [65]:
count_true = 0
for i in range(len(data[:10])):
    text = data[i]['text']
    pred = str(pip_2(text)[0]['word'])
    print(pred)


Размер обеспечения исполнения контракта 6593. 25 Российский рубль
должен предоставить обеспечение исполнения контракта в размере 10 % от цены Контракта.
исполнения контракта
исполнения контракта
00 %
Размер обеспечения исполнения контракта устанавливается в размере 5 ( пять ) процентов от цены, по которой заключается контракт и составляет _ _ _ _ _ _ _ _ _
Размер обеспечение исполнения контракта устанавливается от 0, 5 % до 30 % начальной ( максимальной ) цены контракта
обеспечения
Размер обеспечение исполнения контракта устанавливается от 0, 5 % до 30 % начальной ( максимальной ) цены контракта
Размер обеспечение исполнения контракта устанавливается от 0, 5 % до 30 % начальной ( максимальной ) цены контракта
